In [2]:
%pip install pandas numpy loguru requests


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import pandas as pd
import json
import numpy as np

try:
    HERE = Path(__file__).resolve()
except NameError:
    HERE = Path.cwd()

data = pd.read_csv(HERE.parent / "gemma_annotation_normalized.csv", dtype=str)

In [28]:
from loguru import logger

logger.add("API_Errors.log", level = "INFO", format = "{time}:{level}:{message}")

5

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         910 non-null    object
 1   ingredient         910 non-null    object
 2   amount             910 non-null    object
 3   ingr_annotation    910 non-null    object
 4   amount_annotation  910 non-null    object
 5   norm_value         910 non-null    object
 6   norm_unit          910 non-null    object
dtypes: object(7)
memory usage: 49.9+ KB


In [30]:
import requests

url = "https://smarthome.uni-regensburg.de/naehrwertrechner/api/1.0/recipe_info_optifast"

def get_nut(prompt: str) -> dict | None:
    payload = { "recipe": prompt }
    headers = {"content-type": "application/json"}
    response = requests.post(url, json = payload, headers = headers, verify=False)

    if response.ok:
        return response.json()
    else:
        return None

In [ ]:
import json

def is_invalid(res: dict) -> bool:
    """
        check if the response from the API is valid or not.
        If the output is "Nicht Erkannt" then this function would returns True
        And log the error in a seperate file when an error occurs at the API
    """
    if not res or not res["detailed_info"] or res["detailed_info"][0][0]["erkannteZutat"] == "Nicht erkannt":
        return True
    return False

def calc_nut(x: any) -> str:
    """
        calculate the nutrition value using ingredient name and quantity.
        If the output is "Nicht Erkannt" then this function would returns np.NaN
        And log the error in a seperate file when an error occurs at the API
    """
    amount, unit  = x["norm_value"], x["norm_unit"]

    # pick up the ingredient from zutat
    ingr = json.loads(x["amount_annotation"])["zutat"]

    prompt = f"{amount} {unit} {ingr}"

    print(prompt)

    res = get_nut(prompt)

    if is_invalid(res):
        ingr = x["ingredient"]

        new_prompt = f"{amount} {unit} {ingr}"

        print(f'fallback prompt: {new_prompt}')

        res = get_nut(new_prompt)

        if is_invalid(res):
            logger.error(f"prompt1: {prompt}, prompt2: {new_prompt}")
            return None

    try:
        return json.dumps(res)
    except TypeError as e:
        return None

In [32]:
data['nutrition'] = data.apply(
    lambda x: calc_nut(x),
    axis = 1,
)

1.0 stück Zucchini
2.0 stück Paprika
200.0 g Feta
250.0 g Schinken
1.0 stück Zwiebel
300.0 g Tomate
500.0 g Hähnchenbrust
120.0 g Zwiebel
300.0 g Austernpilz
0.25 liter Schmand
100.0 g Frischkäse
0.015 liter Öl
1.0 stück Butter
0.015 liter Tomatenmark
0.005 liter Majoran
4.0 stück Hähnchenschlegel


2025-11-09 20:33:38.786 | ERROR    | __main__:calc_nut:40 - prompt1: 4.0 stück Hähnchenschlegel, prompt2: 4.0 stück bio hähnchenschlegel


fallback prompt: 4.0 stück bio hähnchenschlegel
2.0 stück Hähnchen
0.02 liter Schuss
fallback prompt: 0.02 liter rapsöl
2.0 stück Zwiebel
190.0 g Rosenkohl
80.0 g Karotte
40.0 g Lauch
0.1 liter Wein
150.0 g Champignon
0.2 liter Sahne
0.005 liter Butter
0.005 liter Mehl
150.0 g Frischkäse
0.03 liter Milch
0.015 liter Zucker
1.0 stück Heidelbeere
1.0 stück Schokoriegel


2025-11-09 20:33:42.548 | ERROR    | __main__:calc_nut:40 - prompt1: 1.0 stück Schokoriegel, prompt2: 1.0 stück schokoriegel


fallback prompt: 1.0 stück schokoriegel
0.03 liter Amarettini


...

30.0 g Brokkoli
30.0 g Spitzpaprika
fallback prompt: 30.0 g spitzpaprika rot


2025-11-09 20:37:05.553 | ERROR    | __main__:calc_nut:40 - prompt1: 30.0 g Spitzpaprika, prompt2: 30.0 g spitzpaprika rot


0.015 liter Kraut
0.0005 liter Prise
fallback prompt: 0.0005 liter meersalz
0.0005 liter Cayenne
6.0 stück Schinken
6.0 stück Radieschen
0.015 liter Speiseöl
fallback prompt: 0.015 liter speiseöl


2025-11-09 20:37:07.226 | ERROR    | __main__:calc_nut:40 - prompt1: 0.015 liter Speiseöl, prompt2: 0.015 liter speiseöl


0.005 liter Essig
1.0 liter Milch
250.0 g Milchreis
0.0005 liter Salz
0.03 liter Zucker


In [37]:
# calculate the number of `Nicht Erkannt`
data["nutrition"].isna().sum()

np.int64(73)

In [ ]:
# Filter out rows where nutrition is None
filtered = data[data['nutrition'].isna()]

# Save to CSV
filtered.to_csv("failed_extracts.csv", index=False)
filtered.head(10)

In [38]:
# remove rows with error cases
data.dropna(subset=["nutrition"])

,Unnamed: 0,ingredient,amount,ingr_annotation,amount_annotation,norm_value,norm_unit,nutrition
0,0,zucchini,1,"{""anzahl"": 1, ""einheit"": ""St\u00fcck""}","{""zutat"": ""Zucchini""}",1.0,stück,"{""detailed_info"": [[{""menge"": ""175.0g"", ""einhe..."
1,1,paprika rot,2,"{""anzahl"": 2, ""einheit"": ""St\u00fcck""}","{""zutat"": ""Paprika"", ""eigenschaft"": ""rot""}",2.0,stück,"{""detailed_info"": [[{""menge"": ""0.0g"", ""einheit..."
2,2,feta,200 g,"{""gewicht"": 200, ""einheit"": ""g""}","{""zutat"": ""Feta""}",200.0,g,"{""detailed_info"": [[{""menge"": ""200.0g"", ""einhe..."
3,3,schinken,250 g,"{""gewicht"": 250, ""einheit"": ""g""}","{""zutat"": ""Schinken""}",250.0,g,"{""detailed_info"": [[{""menge"": ""250.0g"", ""einhe..."
4,4,zwiebel frisch,1,"{""anzahl"": 1, ""einheit"": ""St\u00fcck""}","{""zutat"": ""Zwiebel"", ""eigenschaft"": ""frisch""}",1.0,stück,"{""detailed_info"": [[{""menge"": ""100.0g"", ""einhe..."
...,...,...,...,...,...,...,...,...
905,905,weinessig,1 tl,"{""volumen"": 1, ""einheit"": ""tl""}","{""zutat"": ""Essig"", ""eigenschaft"": ""wei\u00df""}",0.005,liter,"{""detailed_info"": [[{""menge"": ""10.0g"", ""einhei..."
906,906,milch,1 liter,"{""volumen"": 1, ""einheit"": ""liter""}","{""zutat"": ""Milch"", ""eigenschaft"": ""1 Liter""}",1.0,liter,"{""detailed_info"": [[{""menge"": ""1000.0g"", ""einh..."
907,907,milchreis,250 g,"{""gewicht"": 250, ""einheit"": ""g""}","{""zutat"": ""Milchreis""}",250.0,g,"{""detailed_info"": [[{""menge"": ""250.0g"", ""einhe..."
908,908,salz,1 prise,"{""anzahl"": 1, ""einheit"": ""Prise"", ""zutat"": ""sa...","{""zutat"": ""Salz"", ""eigenschaft"": ""Prise""}",0.0005,liter,"{""detailed_info"": [[{""menge"": ""1000.0g"", ""einh..."


In [ ]:
# saving it this way would still keep ingredient rows with none as value for nutrition part...look at output
data.to_csv(HERE.parent / "gemma_annotation_nutrition.csv", index = False)